## 3.4 Regional Planning
The SOUTHERN CONFEDERATION OF KIBBUTZIM is a group of three kibbutzim (communal farming communities) in Israel. Overall planning for this group is done in its Coordinating Technical Office. This office currently is planning agricultural production for the coming year.

The agricultural output of each kibbutz is limited by both the amount of available irrigable land and the quantity of water allocated for irrigation by the Water Commissioner (a national government official). These data are given in Table 3.8. The crops suited for this region include sugar beets, cotton, and sorghum, and these are the three being considered for the upcoming season. These crops differ primarily in their expected net return per acre and their consumption of water. In addition, the Ministry of Agriculture has set a maximum quota for the total acreage that can be devoted to each of these crops by the Southern Confederation of Kibbutzim, as shown in Table 3.9.

---

 **Formulation as a Linear Programming Problem**

The quantities to be decided upon are the number of acres to devote to each of the three crops at each of the three kibbutzim. The decision variables $x_j$ (j = 1, 2, . . . , 9) represent these nine quantities.

Since the measure of effectiveness $Z$ is the total net return, the resulting linear programming model for this problem is:

**Maximize**
$$Z = 1,000(x_1 + x_2 + x_3) + 750(x_4 + x_5 + x_6) + 250$$


**TABLE 3.10 Decision variables for the Southern Confederation of Kibbutzim problem**

| Crop | Kibbutz 1 | Kibbutz 2 | Kibbutz 3 | Net Return | Max Output | Water Consumption |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| Sugar beets | $x_1$ | $x_2$ | $x_3$ | $1,000$ | 600 | 3 |
| Cotton | $x_4$ | $x_5$ | $x_6$ | $750$ | 500 | 2 |
| Sorghum | $x_7$ | $x_8$ | $x_9$ | $250$ | 325 | 1 |
|---|
| Water Allocation | 600 | 800 | 350 |
| Usable Land | 400 | 600 | 300 |



In [10]:
using JuMP, GLPK

model = Model(GLPK.Optimizer)

@variable(model, x[1:9] >= 0)

9-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]

In [11]:
# We want to maximize the total net return
@objective(model, Max, 1000 * (x[1] + x[2] + x[3]) + 750 * (x[4] + x[5] + x[6]) + 250 * (x[7] + x[8] + x[9]))

1000 x[1] + 1000 x[2] + 1000 x[3] + 750 x[4] + 750 x[5] + 750 x[6] + 250 x[7] + 250 x[8] + 250 x[9]

In [12]:
# Constraints for water allocation for each kibbutz
@constraint(model, 3 * x[1] + 2 * x[4] + 1 * x[7] <= 600) # Kibbutz 1
@constraint(model, 3 * x[2] + 2 * x[5] + 1 * x[8] <= 800) # Kibbutz 2
@constraint(model, 3 * x[3] + 2 * x[6] + 1 * x[9] <= 375) # Kibbutz 3

# Constraints for usable land
@constraint(model, x[1] + x[4] + x[7] <= 400) # Kibbutz 1
@constraint(model, x[2] + x[5] + x[8] <= 600) # Kibbutz 2
@constraint(model, x[3] + x[6] + x[9] <= 300) # Kibbutz 3

# Maximum output for each crop
@constraint(model, x[1] + x[2] + x[3] <= 600)
@constraint(model, x[4] + x[5] + x[6] <= 500)
@constraint(model, x[7] + x[8] + x[9] <= 325)

x[7] + x[8] + x[9] ≤ 325

In [13]:
# Since proportion of each crop is 1:1:1, we can use the following constraint
@constraint(model, 3*(x[1] + x[4] + x[7]) - 2*(x[2] + x[5] + x[8]) == 0)
@constraint(model, 1*(x[2] + x[5] + x[8]) - 2*(x[3] + x[6] + x[9]) == 0)
@constraint(model, 4*(x[3] + x[6] + x[9]) - 3*(x[1] + x[4] + x[7]) == 0)

-3 x[1] + 4 x[3] - 3 x[4] + 4 x[6] - 3 x[7] + 4 x[9] = 0

In [14]:
# Print the objective
println("Objective: ", objective_sense(model), " ", objective_function(model))
println("-"^30) # A separator line

# Print each constraint on a new line
println("Constraints:")
for con_ref in all_constraints(model)
    println("  ", name(con_ref), ": ", constraint_object(con_ref))
end

Objective: MAX_SENSE 1000 x[1] + 1000 x[2] + 1000 x[3] + 750 x[4] + 750 x[5] + 750 x[6] + 250 x[7] + 250 x[8] + 250 x[9]
------------------------------
Constraints:


UndefKeywordError: UndefKeywordError: keyword argument `include_variable_in_set_constraints` not assigned

In [15]:
# Finally, we solve the model
optimize!(model)

println("Optimal solution: $(objective_value(model))\n")

# Preview the results
for i in 1:9
    println("x[$i] = ", value(x[i]))
end

Optimal solution: 633333.3333333333

x[1] = 133.33333333333331
x[2] = 100.0
x[3] = 25.0
x[4] = 100.00000000000001
x[5] = 250.0
x[6] = 150.0
x[7] = 0.0
x[8] = 0.0
x[9] = 0.0


In [16]:
max_z = objective_value(model)
println("Max Z = ", max_z)

Max Z = 633333.3333333333
